# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,14.89,72,100,11.58,ZA,2023-06-12
1,1,todos santos,23.4500,-110.2167,19.47,64,0,3.30,MX,2023-06-12
2,2,port lincoln,-34.7333,135.8667,13.57,70,76,8.93,AU,2023-06-12
3,3,acajutla,13.5928,-89.8275,26.98,87,97,2.82,SV,2023-06-12
4,4,iqaluit,63.7506,-68.5145,0.85,80,75,6.71,CA,2023-06-12


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    # geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    scale = 0.9,
    color = "City"
)  

# Display the map plot
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_warm = city_data_df[(city_data_df["Max Temp"] > 20)]

# Drop any rows with null values
my_city_df = city_data_warm.dropna(how='any')

# Display sample data
my_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,acajutla,13.5928,-89.8275,26.98,87,97,2.82,SV,2023-06-12
9,9,telese,41.2173,14.5277,27.45,64,34,0.78,IT,2023-06-12
10,10,palikir - national government center,6.9248,158.1611,28.41,79,100,2.57,FM,2023-06-12
11,11,adamstown,-25.0660,-130.1015,20.24,63,38,4.76,PN,2023-06-12
12,12,afaahiti,-17.7500,-149.2833,21.93,66,90,7.24,PF,2023-06-12
...,...,...,...,...,...,...,...,...,...,...
592,592,karema,-6.8205,30.4389,28.11,51,11,4.03,TZ,2023-06-12
593,593,jinka,5.6500,36.6500,24.32,62,79,1.40,ET,2023-06-12
595,595,mabaruma,8.2000,-59.7833,26.60,90,98,1.26,GY,2023-06-12
596,596,belmonte,-15.8631,-38.8828,26.61,65,13,2.59,BR,2023-06-12


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_city_df.copy(deep=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,acajutla,13.5928,-89.8275,26.98,87,97,2.82,SV,2023-06-12,
9,9,telese,41.2173,14.5277,27.45,64,34,0.78,IT,2023-06-12,
10,10,palikir - national government center,6.9248,158.1611,28.41,79,100,2.57,FM,2023-06-12,
11,11,adamstown,-25.0660,-130.1015,20.24,63,38,4.76,PN,2023-06-12,
12,12,afaahiti,-17.7500,-149.2833,21.93,66,90,7.24,PF,2023-06-12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    city = row["City"]

    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
acajutla - nearest hotel: Hotel Lara
telese - nearest hotel: Albergo D'Onofrio
palikir - national government center - nearest hotel: Mangrove Bay Hotel
adamstown - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
west island - nearest hotel: Cocos Village Bungalows
hithadhoo - nearest hotel: Pebbles Inn
marine corps base hawaii - mcbh - nearest hotel: No hotel found
nchelenge - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
ust'-talovka - nearest hotel: Металлург
puerto baquerizo moreno - nearest hotel: Dory's House
ciechocinek - nearest hotel: Pałac Łazienki II
rongelap - nearest hotel: No hotel found
beitbridge - nearest hotel: Beitbridge Inn & Casino
uturoa - nearest hotel: Hawaiki Nui hotel
gizo - nearest hotel: Gizo Hotel
banda aceh - nearest hotel: Hotel 61
dhari - nearest hotel: No hotel found
puerto armuelles - nearest hotel: Coco del Mar
richards bay - nearest hotel: Bon Hotel Waterfront
sumbe - nearest hotel: N

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    scale = 0.9,
    color = "City",
    hover_cols = ['Country', 'Hotel Name'],
)  

# Display the map

map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)